# Spam Classifier
## Assignment Preamble
Please ensure you carefully read all of the details and instructions on the assignment page, this section, and the rest of the notebook. If anything is unclear at any time please post on the forum or ask a tutor well in advance of the assignment deadline.

In addition to all of the instructions in the body of the assignment below, you must also follow the following technical instructions for all assignments in this unit. *Failure to do so may result in a grade of zero.*
* [At the bottom of the page](#Submission-Test) is some code which checks you meet the submission requirements. You **must** ensure that this runs correctly before submission.
* Do not modify or delete any of the cells that are marked as test cells, even if they appear to be empty.
* Do not duplicate any cells in the notebook – this can break the marking script. Instead, insert a new cell (e.g. from the menu) and copy across any contents as necessary.

Remember to save and backup your work regularly, and double-check you are submitting the correct version.

This notebook is the primary reference for your submission. You may write code in separate `.py` files but it must be clearly imported into the notebook so that it runs without needing to reference those files, and you must explain clearly what functionality is contained in those files (through comments, markdown cells, etc).

As always, **the work you submit for this assignment must be entirely your own.** Do not copy or work with other students. Do not copy answers that you find online. These assignments are designed to help improve your understanding first and foremost – the process of doing the assignment is part of *learning*. They are also used to assess your ability, and so you must uphold academic integrity. Submitting plagiarised work risks your entire place on your degree.

**The pass mark for this assignment is 40%.** We expect that students, on average, will be able to produce a submission which gets a mark between 50-70% within the normal workload allocation for the unit, but this will vary depending on individual backgrounds. Please ask for help if you are struggling.

## Getting Started
Spam refers to unwanted email, often in the form of advertisements. In the literature, an email that is **not** spam is called *ham*. Most email providers offer automatic spam filtering, where spam emails will be moved to a separate inbox based on their contents. Of course this requires being able to scan an email and determine whether it is spam or ham, a classification problem. This is the subject of this assignment.

This assignment has two parts. Each part is worth 50% of the overall grade for this assignment.

For part one you will write a supervised learning based classifier to determine whether a given email is spam or ham. You must write and submit the code in this notebook. The training data is provided for you. You may use any classification method. Marks will be awarded primarily based on the accuracy of your classifier on unseen test data, but there are also marks for estimating how accurate you think your classifier will be.

In part two you will produce a short video explaining your implementation, any decisions or extensions you made, and what parameter values you used. This part is explained in more detail on the assignment page. The video file must be submitted with your assignment.

### Choice of Algorithm
While the classification method is a completely free choice, the assignment folder includes [a separate notebook file](data/naivebayes.ipynb) which can help you implement a Naïve Bayes solution. If you do use this notebook, you are still responsible for porting your code into *this* notebook for submission. A good implementation should give a high  enough accuracy to get a good grade on this section (50-70%).

You could also consider a k-nearest neighbour algorithm, but this may be less accurate. Logistic regression is another option that you may wish to consider.

If you are looking to go beyond the scope of the unit, you might be interested in building something more advanced, like an artificial neural network. This is possible just using `numpy`, but will require significant self-directed learning. *Extensions like this are left unguided and are not factored into the unit workload estimates.*

**Note:** you may use helper functions in libraries like `numpy` or `scipy`, but you **must not** import code which builds entire models for you. This includes but is not limited to use of libraries like `scikit-learn`, `tensorflow`, or `pytorch` – there will be plenty of opportunities for these libraries in later units. The point of this assignment is to understand code the actual algorithm yourself. ***If you are in any doubt about any particular library or function please ask a tutor.*** Submissions which ignore this will receive penalties or even zero marks.

If you choose to implement more than one algorithm, please feel free to include your code and talk about it in part two (your video presentation), but only the code in this notebook will be used in the automated testing.

## Training Data
The training data is described below and has 1000 rows. There is also a 500 row set of test data. These are functionally identical to the training data, they are just in a separate csv file to encourage you to split out your training and test data. You should consider how to best make use of all available data without overfitting, and to help produce an unbiased estimate for your classifier's accuracy.

The cell below loads the training data into a variable called `training_spam`.

In [1]:
import numpy as np

training_spam = np.loadtxt(open("data/training_spam.csv"), delimiter=",").astype(int)
print("Shape of the spam training data set:", training_spam.shape)
print(training_spam)

Shape of the spam training data set: (1000, 55)
[[1 0 0 ... 0 0 0]
 [0 0 1 ... 1 0 0]
 [0 0 0 ... 1 0 0]
 ...
 [0 0 0 ... 0 0 1]
 [1 1 1 ... 1 1 0]
 [1 0 0 ... 1 1 1]]


Your training set consists of 1000 rows and 55 columns. Each row corresponds to one email message. The first column is the _response_ variable and describes whether a message is spam `1` or ham `0`. The remaining 54 columns are _features_ that you will use to build a classifier. These features correspond to 54 different keywords (such as "money", "free", and "receive") and special characters (such as ":", "!", and "$"). A feature has the value `1` if the keyword appears in the message and `0` otherwise.

As mentioned there is also a 500 row set of *test data*. It contains the same 55 columns.

In [2]:
testing_spam = np.loadtxt(open("data/testing_spam.csv"), delimiter=",").astype(int)
print("Shape of the spam testing data set:", testing_spam.shape)
print(testing_spam)

Shape of the spam testing data set: (500, 55)
[[1 0 0 ... 1 1 1]
 [1 1 0 ... 1 1 1]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 1 0 0]
 [0 0 0 ... 1 0 0]]


## Part One
Write all of the code for your classifier below this cell. There is some very rough skeleton code in the cell directly below. You may insert more cells below this if you wish, but you must not duplicate any cells as this can break the grading script.

### Submission Requirements
Your code must provide a variable with the name `classifier`. This object must have a method called `predict` which takes input data and returns class predictions. The input will be a single $n \times 54$ numpy array, your classifier should return a numpy array of length $n$ with classifications. There is a demo in the cell below, and a test you can run before submitting to check your code is working correctly.

Your code must run on our test machine in under 30 seconds. If you wish to train a more complicated model (e.g. neural network) which will take longer, you are welcome to save the model's weights as a file and then load these in the cell below so we can test it. You must include the code which computes the original weights, but this must not run when we run the notebook – comment out the code which actually executes the routine and make sure it is clear what we need to change to get it to run. Remember that we will be testing your final classifier on additional hidden data.

In [3]:
import numpy as np
import math


class SpamClassifier:
    def __init__(self, k):
        self.log_class_conditional_likelihoods = None
        self.log_class_priors = None
        print("Classifier is initialising")
        self.k = k
        self.training_spam = np.loadtxt(open("data/training_spam.csv"), delimiter=",").astype(int)
        self.decision_tree = np.empty(0)

        print("Shape of the spam training data set:", self.training_spam.shape)
        # print(self.training_spam)

    def train(self, use_decision_tree=1):
        if use_decision_tree == 0:
            print("Training naive bayes against training data")
            self.estimate_log_class_priors(self.training_spam)
            self.estimate_log_class_conditional_likelihoods(self.training_spam)
        elif use_decision_tree == 1:
            print("Training decision tree")
            self.decision_tree = self.train_id3_decision_tree(self.training_spam)
        else:
            print("Training decision tree")
            self.decision_tree = self.train_id3_decision_tree(self.training_spam)
            print("Parsing decision tree against training data")
            decision_tree_feedback = self.parse_decision_tree(self.training_spam)
            print("Appending decision tree predictions to training data")
            enhanced_train_data = np.zeros(self.training_spam.shape[1] + 1)
            for x in zip(self.training_spam, decision_tree_feedback):
                # print("combined row: ", np.append(x[0], x[1]))
                enhanced_train_data = np.vstack((enhanced_train_data, np.append(x[0], x[1])))
            enhanced_train_data = np.delete(enhanced_train_data, (0), axis=0)
            print("Enhanced training data shape: ", enhanced_train_data.shape)
            print("Training naive bayes against enhanced data")
            self.estimate_log_class_priors(enhanced_train_data)
            self.estimate_log_class_conditional_likelihoods(enhanced_train_data)
            print("Naive bayes trained against enhanced data")

    def estimate_log_class_priors(self, data):
        """
        Given a data set with binary response variable (0s and 1s) in the
        left-most column, calculate the logarithm of the empirical class priors,
        that is, the logarithm of the proportions of 0s and 1s:
            log(p(C=0)) and log(p(C=1))

        :param data: a two-dimensional numpy-array with shape = [n_samples, 1 + n_features]
                     the first column contains the binary response (coded as 0s and 1s).

        :return log_class_priors: a numpy array of length two
        """

        num_rows = data.shape[0] + 1  # +1 for laplace smoothing
        count_c1 = np.sum(data[:, 0]) + 1

        log_c0_prior = math.log((num_rows - count_c1) / num_rows)
        log_c1_prior = math.log(count_c1 / num_rows)
        log_class_priors = np.array([log_c0_prior, log_c1_prior])

        self.log_class_priors = log_class_priors

    def estimate_log_class_conditional_likelihoods(self, data, alpha=1.0):
        """
        Given a data set with binary response variable (0s and 1s) in the
        left-most column and binary features (words), calculate the empirical
        class-conditional likelihoods, that is,
        log(P(w_i | c)) for all features w_i and both classes (c in {0, 1}).

        Assume a multinomial feature distribution and use Laplace smoothing
        if alpha > 0.

        :param data: a two-dimensional numpy-array with shape = [n_samples, 1 + n_features]
        :param alpha: indicator used to determine whether laplace smoothing is used as per
                        project requirements

        :return theta:
            a numpy array of shape = [2, n_features]. theta[j, i] corresponds to the
            logarithm of the probability of feature i appearing in a sample belonging
            to class j.
        """
        # put together to create a 1D array of the sum of each element where C=1 (tested and correct in excel)
        c1_rows = data[np.nonzero(data[:, 0]), 1:]
        c1_numrows = c1_rows.shape[1]
        c1_element_sums = np.array(np.sum(c1_rows[0, :], axis=0))
        if alpha > 0:
            c1_element_sums = np.add(1, c1_element_sums)  # add 1 to every element for laplace smoothing
        c1_element_eccl = np.log(np.divide(c1_element_sums, c1_numrows))

        # now get rows where C=0 using idea from 
        #   https://stackoverflow.com/questions/4588628/find-indices-of-elements-equal-to-zero-in-a-numpy-array
        c0_rows = data[np.where(data[:, 0] == 0)[0], 1:]
        c0_numrows = c0_rows.shape[0]
        c0_element_sums = np.array(np.sum(c0_rows, axis=0))
        if alpha > 0:
            c0_element_sums = np.add(1, c0_element_sums)  # add 1 to every element for laplace smoothing
        c0_element_eccl = np.log(np.divide(c0_element_sums, c0_numrows))

        theta = np.array([c0_element_eccl, c1_element_eccl])

        self.log_class_conditional_likelihoods = theta

    def get_probability(self, class_num, message, log_class_priors, log_class_conditional_likelihoods):
        running_probability = 1
        i = 0
        for element in message:
            if element == 1:
                running_probability += log_class_conditional_likelihoods[class_num, i]
            i += 1

        running_probability *= log_class_priors[class_num]
        return running_probability

    def predict(self, data, use_decision_tree=1):
        """
        Given a new data set with binary features, predict the corresponding
        response for each instance (row) of the new_data set.

        :param use_decision_tree: integer value to decide what form of test to run (must match
            the form chosen in train method)
            0 = Naive Bayes
            1 = Decision Tree (default)
            2 = Decision Tree and Naive Bayes
        :param data: a two-dimensional numpy-array with shape = [n_test_samples, n_features].

        :return class_predictions: a numpy array containing the class predictions for each row
            of new_data.
        """

        self.train(use_decision_tree)
        class_predictions = np.empty(data.shape[0])
        message = 0

        if use_decision_tree == 0:
            print("Running naive bayes against test data")
            class_predictions = self.run_naive_bayes(data)
        elif use_decision_tree == 1:
            print("Parsing decision tree against test data")
            class_predictions = self.parse_decision_tree(data)
        else:
            print("Parsing decision tree against test data")
            class_predictions = self.parse_decision_tree(data)
            print("Adding decision tree predictions to test data")
            enhanced_test_data = np.zeros(data.shape[1] + 1)
            for x in zip(data, class_predictions):
                enhanced_test_data = np.vstack((enhanced_test_data, np.append(x[0], x[1])))
            enhanced_test_data = np.delete(enhanced_test_data, (0), axis=0)
            print("Running naive bayes against enhanced test data")
            class_predictions = self.run_naive_bayes(enhanced_test_data)

        return class_predictions

    def train_id3_decision_tree(self, training_spam):
        # 'ID3 decision tree generator
        # ref https://medium.com/analytics-vidhya/entropy-calculation-information-gain-decision-tree-learning-771325d16f'
        training_spam = np.vstack(([i for i in range(-1, training_spam.shape[1] - 1)],
                                   training_spam))  # add index ref header row
        return self.generate_decision_tree(training_spam)

    def generate_decision_tree(self, training_spam):
        training_spam = np.unique(training_spam, axis=0)  # remove duplicate rows
        gain_per_attribute = np.zeros((training_spam.shape[1] - 1, 2))
        max_gain = np.zeros(3)  # array used to store running highest gain
        #                                format: [index in current S, index in original, gain]
        # print("calculating max gain for table:\n", training_spam)
        for S in range(1, training_spam.shape[1]):
            # print("checking index ", S)
            s0c0 = s0c1 = s1c0 = s1c1 = s0 = s1 = c0 = c1 = 0
            for i in range(1, training_spam.shape[0] - 1):
                if training_spam[i][S] == 0:
                    if training_spam[i][0] == 0:
                        s0c0 += 1
                    else:  # c=1
                        s0c1 += 1
                else:  # S=1
                    if training_spam[i][0] == 0:
                        s1c0 += 1
                    else:  # c=1
                        s1c1 += 1
            s0 = s0c0 + s0c1
            s1 = s1c0 + s1c1
            c0 = s0c0 + s1c0
            c1 = s0c1 + s1c1
            total = c0 + c1

            # Decide ham or spam based on majority rules if only 1 column at this point
            if training_spam.shape[1] == 2:
                return c1 > c0  #

            # return terminals if all classifications are the same
            if c0 == 0:
                return True  # "SPAM"
            if c1 == 0:
                return False  # "HAM"

            s_gain = self.calculate_gain(s0c0, s0c1, s1c0, s1c1, s0, s1, c0, c1, total)
            # print("Gain for function column {} = {}".format(S, s_gain))

            if s_gain > max_gain[2]:
                max_gain = [S, training_spam[0, S], s_gain]
            gain_per_attribute[S - 1] = [training_spam[0, S], s_gain]
        # print("about to split data - attribute with max gain = {} in index {}".format(max_gain[1], max_gain[0]))

        # if there is no entropy on any attribute (for example all attributes have zero
        #       positive values), no gain will have been selected - just select the first attribute.
        if max_gain[0] == 0.0:
            max_gain = [1, training_spam[0, 1], 0]

        s0_data = self.split_on_attribute_value(training_spam, max_gain[0], 0)
        s1_data = self.split_on_attribute_value(training_spam, max_gain[0], 1)

        branch = [max_gain[1], self.generate_decision_tree(s0_data),
                  self.generate_decision_tree(s1_data)]
        return branch

    def calculate_gain(self, s0c0, s0c1, s1c0, s1c1, s0, s1, c0, c1, total):

        # print("s0({}) = s0c0({}) + s0c1({})".format(s0, s0c0, s0c1))
        # print("s1({}) = s1c0({}) + s1c1({})".format(s1, s1c0, s1c1))
        # print("c0({}) = s0c0({}) + s1c0({})".format(c0, s0c0, s1c0))
        # print("c1({}) = s0c1({}) + s1c1({})".format(c1, s0c1, s1c1))

        # lambda to protect against dividing by or applying log2 to zero,
        #       confirms neither side is zero then returns entropy instance calculation
        entropy_instance = lambda x, y: 0 if x == 0 or y == 0 else (x / y) * math.log2(x / y)

        # entropy = (c0 / total) * math.log2(c0 / total) + (c1 / total) * math.log2(c1 / total)
        entropy = -(entropy_instance(c0, total) + entropy_instance(c1, total))

        # entropy_s0 = -((s0c0 / s0) * math.log2(s0c0 / s0) + (s0c1 / s0) * math.log2(s0c1 / s0))
        entropy_s0 = -(entropy_instance(s0c0, s0) + entropy_instance(s0c1, s0))

        # entropy_s1 = -((s1c0 / s0) * math.log2(s1c0 / s0) + (s1c1 / s0) * math.log2(s1c1 / s0))
        entropy_s1 = -(entropy_instance(s1c0, s1) + entropy_instance(s1c1, s1))

        gain = entropy - (((s0 / total) * entropy_s0) + ((s1 / total) * entropy_s1))

        return gain

    def split_on_attribute_value(self, input_table, attribute_index, value):
        # If there are only 3 columns left there are not enough left to split - simply delete the other column
        if input_table.shape[1] == 3:
            result = np.delete(input_table, value + 1, axis=1)
            return result
        # print("input table to split on value {} in attribute{}:\n{}".format(value, attribute_index, input_table))
        result_rows = np.insert(np.where(input_table[:, attribute_index] == value), 0, 0)
        result = np.delete(input_table[result_rows], attribute_index, axis=1)
        return result

    def parse_decision_tree(self, data):
        output = []
        jj = -1
        for row in data:
            jj += 1
            row_output = None
            current_branch = self.decision_tree

            while row_output == None:
                current_attribute = current_branch[0]
                current_value = row[current_attribute]
                if isinstance(current_value, np.int32):
                    if current_value == 0:
                        if isinstance(current_branch[1], bool):
                            row_output = current_branch[1]
                        current_branch = current_branch[1]
                    else:  # ==1
                        if isinstance(current_branch[2], bool):
                            row_output = current_branch[2]
                        else:
                            current_branch = current_branch[2]
                else:
                    row_output = row[current_attribute]
                if row_output is not None:
                    output.append(int(row_output is True))
                    # print("prediction is SPAM: ", row_output)
        return np.asarray(output)

    def run_naive_bayes(self, data):
        class_predictions = np.empty(data.shape[0])
        message = 0

        for row in data:
            # print("Row: ", row)
            class0_probability = self.get_probability(0, row, self.log_class_priors,
                                                      self.log_class_conditional_likelihoods)
            # print("class0_probability: ", class0_probability)
            class1_probability = self.get_probability(1, row, self.log_class_priors,
                                                      self.log_class_conditional_likelihoods)
            # print("class1_probability: ", class1_probability)
            probably_spam = class1_probability < class0_probability
            # print("probably_spam: ", probably_spam)
            class_predictions[message] = probably_spam
            message += 1

        return class_predictions


def create_classifier():
    classifier = SpamClassifier(k=1)
    return classifier


classifier = create_classifier()


Classifier is initialising
Shape of the spam training data set: (1000, 55)


### Accuracy Estimate
In the cell below there is a function called `my_accuracy_estimate()` which returns `0.5`. Before you submit the assignment, write your best guess for the accuracy of your classifier into this function, as a percentage between `0` and `1`. So if you think you will get 80% of inputs correct, return the value `0.8`. This will form a small part of the marking criteria for the assignment, to encourage you to test your own code.

In [4]:
def my_accuracy_estimate():
    return 0.898

Write all of the code for your classifier above this cell.

### Testing Details
Your classifier will be tested against some hidden data from the same source as the original. The accuracy (percentage of classifications correct) will be calculated, then benchmarked against common methods. At the very high end of the grading scale, your accuracy will also be compared to the best submissions from other students (in your own cohort and others!). Your estimate from the cell above will also factor in, and you will be rewarded for being close to your actual accuracy (overestimates and underestimates will be treated the same).

#### Test Cell
The following code will run your classifier against the provided test data. To enable it, set the constant `SKIP_TESTS` to `False`.

The original skeleton code above classifies every row as ham, but once you have written your own classifier you can run this cell again to test it. So long as your code sets up a variable called `classifier` with a method called `predict`, the test code will be able to run. 

Of course you may wish to test your classifier in additional ways, but you *must* ensure this version still runs before submitting.

**IMPORTANT**: you must set `SKIP_TESTS` back to `True` before submitting this file!

In [10]:
SKIP_TESTS = True

if not SKIP_TESTS:
    testing_spam = np.loadtxt(open("data/testing_spam.csv"), delimiter=",").astype(int)
    test_data = testing_spam[:, 1:]
    test_labels = testing_spam[:, 0]

    predictions = classifier.predict(test_data)
    accuracy = np.count_nonzero(predictions == test_labels)/test_labels.shape[0]
    print(f"Accuracy on test data is: {accuracy}")

In [11]:
import sys
import pathlib

fail = False;

if not SKIP_TESTS:
    fail = True;
    print("You must set the SKIP_TESTS constant to True in the cell above.")
    
p3 = pathlib.Path('./spamclassifier.ipynb')
if not p3.is_file():
    fail = True
    print("This notebook file must be named spamclassifier.ipynb")
    
if "create_classifier" not in dir():
    fail = True;
    print("You must include a function called create_classifier.")

if "my_accuracy_estimate" not in dir():
    fail = True;
    print("You must include a function called my_accuracy_estimate.")
else:
    if my_accuracy_estimate() == 0.5:
        print("Warning:")
        print("You do not seem to have provided an accuracy estimate, it is set to 0.5.")
        print("This is the actually the worst possible accuracy – if your classifier")
        print("got 0.1 then it could invert its results to get 0.9!")
    
print("INFO: Make sure you follow the instructions on the assignment page to submit your video.")
print("Failing to include this could result in an overall grade of zero for both parts.")
print()

if fail:
    sys.stderr.write("Your submission is not ready! Please read and follow the instructions above.")
else:
    print("All checks passed. When you are ready to submit, upload the notebook and readme file to the")
    print("assignment page, without changing any filenames.")
    print()
    print("If you need to submit multiple files, you can archive them in a .zip file. (No other format.)")

INFO: Make sure you follow the instructions on the assignment page to submit your video.
Failing to include this could result in an overall grade of zero for both parts.

All checks passed. When you are ready to submit, upload the notebook and readme file to the
assignment page, without changing any filenames.

If you need to submit multiple files, you can archive them in a .zip file. (No other format.)


In [ ]:
# This is a test cell. Please do not modify or delete.